In [25]:
# Set covering problem: you need to select a certain number of blocks in order to cover all the initial sets
# Example: find the minimum number of items that together guarantee to cover something
# Definitions

# STATE = contains all the relevant informations to represent the current situation

In [26]:
from random import random
import numpy as np
from functools import reduce
from collections import namedtuple
from queue import PriorityQueue, SimpleQueue, LifoQueue
from math import ceil

In [27]:
PROBLEM_SIZE = 5
NUM_SETS = 10

In [28]:
sets = tuple(np.array([random() < .2 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS))
State = namedtuple('State', ['taken', 'not_taken'])

"""
PROBLEM_SIZE = 3
NUM_SETS = 5
sets = ([False, False, False],
       [False, False, True],
       [False, False, False],
       [True, False, False],
       [True, False, True])
"""

'\nPROBLEM_SIZE = 3\nNUM_SETS = 5\nsets = ([False, False, False],\n       [False, False, True],\n       [False, False, False],\n       [True, False, False],\n       [True, False, True])\n'

In [29]:
def goal_check(state):
  #np.all = checks whether all elements are TRUE
  #reduce = applies the logical or to all elements of the specified iterable
  return np.all(reduce(np.logical_or, [sets[i] for i in state.taken], np.array([False for _ in range(PROBLEM_SIZE)])))

In [30]:
assert(goal_check(State(set(range(NUM_SETS)), set())), "Problem not solvable")

<>:1: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:1: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:1: SyntaxWarning: assertion is always true, perhaps remove parentheses?
C:\Users\Lorenzo\AppData\Local\Temp/ipykernel_18192/987543073.py:1: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(goal_check(State(set(range(NUM_SETS)), set())), "Problem not solvable")


In [31]:
from collections import namedtuple
State = namedtuple('State', ['taken', 'not_taken'])

In [32]:
# A* algorithm implementation

def is_problem_solvable(sets):
    # this function checks that each individual point is covered by at least one set
    row = False
    for j in range(0, PROBLEM_SIZE):
        for i in range(0, NUM_SETS):
            row |= sets[i][j]
        if row == False:
            return False
        else:
            row = False
    return True

def goal_check(state):
    # This is a binary output: did we reach a solution? True of False
    return np.all(reduce(
        np.logical_or,
        [sets[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    ))

def count_covered(state):
    return reduce(
        np.logical_or,
        [sets[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    )

def h(state):
    # optimistic heuristic: let's count the points we are missing to cover over
    # the largest set (where largest means the set with the greatest amount of trues) 
    missing_points = PROBLEM_SIZE - sum(count_covered(state))
    largest_set = np.max(np.array(sets).sum(axis=1))
    # That is: how many sets are needed (optimistically) to cover the universe?
    return ceil(missing_points/largest_set)

def h2(state):
    pass

def g(state):
    return len(state.taken)

def f(state):
    return g(state) + h(state)

def print_sets():
  print("PRINTING SETS:")
  for s in range(0,NUM_SETS):
    print(f"{s}) {sets[s]}")
  print("-"*25)

print_sets()
if(not is_problem_solvable(sets)):
  print("[!] This configuration of the problem is not solvable! Some point is not covered by any set.")
else: 
    frontier = PriorityQueue()
    state = State(set(), set(range(NUM_SETS)))
    frontier.put((f(state), state))

    counter = 0
    _, current_state = frontier.get()
    while not goal_check(current_state):
        counter += 1
        for action in current_state[1]:
            new_state = State(
                current_state.taken ^ {action},
                current_state.not_taken ^ {action},
            )
            print(f"Putting {f(new_state)} with action {new_state}")
            frontier.put((f(new_state), new_state))
        _, current_state = frontier.get()
    print(
        f"Solved in {counter:,} steps ({len(current_state.taken)} tiles. Sets used to cover {current_state.taken})"
    )

PRINTING SETS:
0) [False  True False False False]
1) [False False False False  True]
2) [False False False  True False]
3) [False  True  True  True False]
4) [False False False False False]
5) [ True False False False  True]
6) [False  True False  True  True]
7) [False False False False False]
8) [False False  True False False]
9) [ True  True False False False]
-------------------------
Putting 3 with action State(taken={0}, not_taken={1, 2, 3, 4, 5, 6, 7, 8, 9})
Putting 3 with action State(taken={1}, not_taken={0, 2, 3, 4, 5, 6, 7, 8, 9})
Putting 3 with action State(taken={2}, not_taken={0, 1, 3, 4, 5, 6, 7, 8, 9})
Putting 2 with action State(taken={3}, not_taken={0, 1, 2, 4, 5, 6, 7, 8, 9})
Putting 3 with action State(taken={4}, not_taken={0, 1, 2, 3, 5, 6, 7, 8, 9})
Putting 2 with action State(taken={5}, not_taken={0, 1, 2, 3, 4, 6, 7, 8, 9})
Putting 2 with action State(taken={6}, not_taken={0, 1, 2, 3, 4, 5, 7, 8, 9})
Putting 3 with action State(taken={7}, not_taken={0, 1, 2, 3, 4